In [ ]:
#Task1
# Validation Set-1000 Sentences
#Test Set -1000 Senteces
# Train Set -9L+ Sentences

In [112]:
import pyarrow.parquet as pq
import pandas as pd
def get_data(path,n=1000,threshold=20):
  data=[]
  parquet=pq.ParquetFile(path)
  for batch in parquet.iter_batches(batch_size=1000,columns=["sentence"]):
    df_batch=batch.to_pandas()
    for sentence in df_batch["sentence"]:
      if sentence.endswith("."):
        sentence=sentence[:-1]
      tokens=sentence.split()
      safe=True
      for token in tokens:
        if len(token)>=threshold:
          safe=False
          break
        
      if safe:
        data.append(sentence)
      if len(data)==n:
        return data
  return data


In [113]:
def get_path(file):
  return f"/home/deepakchalla/Desktop/Desktop/NLP/Lab1/{file}.parquet"

In [114]:
train_data=get_data(get_path("train"),n=1000000)
val_data=get_data(get_path("val"))
test_data=get_data(get_path("test"))

In [ ]:
#Task 2
#Get The N-grams for N={1,2,3,4}
from collections import defaultdict
from math import log


In [116]:
def build_ngrams(sentences,n):
    counts=defaultdict(int)
    for sentence in sentences:
      tokens=["<s>"]*(n-1)+sentence.split()+["</s>"]*(min(n-1,1))
      for i in range(len(tokens)-n+1):
        counts[tuple(tokens[i:i+n])]+=1
    
    return counts

In [117]:
def print_top_k_frequencies(self,k=100):
   cnt=0
   print("C(MLE)    Nc    C*")
   print(f'0   0    {self.freq.get(1,0)/len(self.train)}')
   for key,value in sorted(self.freq.items()):
     c=key
     Nc=value
     Nc1=self.freq.get(c+1,0)
     cstar=(c+1)*(Nc1/Nc) if Nc1>0 else c
     print(f"{c}    {Nc}    {cstar}")
     cnt+=1
     if cnt==k:
       break
     

In [118]:
def build_frequencies(counts):
    freq=defaultdict(int)
    for value in counts.values():
      freq[value]+=1
    return freq

In [119]:
def good_turing_pred(self,sentences,V):
    log_prob=[]
    N1=self.freq.get(1,0)
    N=sum(self.train.values())
    p_unseen=log((N1/N)/(V**self.n-N)) if V**self.n>N else log((N1/N))
    for sentence in sentences: 
      tokens=["<s>"]*(self.n-1)+sentence.split()+["</s>"]*(min(self.n-1,1))
      pr=0
      for i in range(len(tokens)-self.n+1):
        n_gram=tuple(tokens[i:i+self.n])
        #Unseen
        if self.train[n_gram]==0:
          pr+=p_unseen
        else:
          c=self.train[n_gram]
          Nc=self.freq.get(c,0)
          Nc1=self.freq.get(c+1,0)
          cstar=(c+1)*(Nc1/Nc) if Nc1>0 else c
          pr+=log(cstar/N)
      log_prob.append(pr)
    return log_prob

In [120]:
class N_Gram:
  def __init__(self,n):
    self.n=n
    self.train=build_ngrams(train_data,self.n)
    self.freq=build_frequencies(self.train)     


In [121]:
unigrams=N_Gram(1)
bigrams=N_Gram(2)
trigrams=N_Gram(3)
quadgrams=N_Gram(4)

In [137]:
V=len(unigrams.train)
uni_test=good_turing_pred(unigrams,test_data,V)
uni_val=good_turing_pred(unigrams,val_data,V)
bi_test=good_turing_pred(bigrams,test_data,V)
bi_val=good_turing_pred(bigrams,val_data,V)
tri_test=good_turing_pred(trigrams,test_data,V)
tri_val=good_turing_pred(trigrams,val_data,V)
quad_test=good_turing_pred(quadgrams,test_data,V)
quad_val=good_turing_pred(quadgrams,val_data,V)

In [127]:
# Backoff
# P=l4P4+l3P3+l2p2+l1p1
# l1+l2+l3+l4=1

In [142]:
import numpy as np
def exp(a):
  return np.exp(a)

In [152]:
#Test Data
lT1,lT2,lT3,lT4=None,None,None,None
best_log_prob=float('-inf')
step_size=(1/10)
#N=100 Time Complexity = (N**n)*1000
#In our case (N**4)*1000= 10^8*1000=10^11
for l1 in np.arange(0,1.0,step_size):
  for l2 in np.arange(0,1.0,step_size):
    for l3 in np.arange(0,1.0,step_size):
      for l4 in np.arange(0,1.0,step_size):
        if l1+l2+l3+l4==1:
          lp=0
          for i in range(len(test_data)):
            l=l1*exp(uni_test[i])+l2*exp(bi_test[i])+l3*exp(tri_test[i])+l4*exp(quad_test[i])
            lp+=log(l) if l>0 else 0
          lp/=1000
          if best_log_prob<lp:
            lT1,lT2,lT3,lT4=l1,l2,l3,l4
            best_log_prob=lp
print(lT1,lT2,lT3,lT4)
print(best_log_prob)


0.9 0.0 0.0 0.1
-108.40513864431794


In [153]:

lV1,lV2,lV3,lV4=None,None,None,None
best_log_prob=float('-inf')
step_size=(1/10)
#N=100 Time Complexity = (N**n)*1000
for l1 in np.arange(0,1.0,step_size):
  for l2 in np.arange(0,1.0,step_size):
    for l3 in np.arange(0,1.0,step_size):
      for l4 in np.arange(0,1.0,step_size):
        if l1+l2+l3+l4==1:
          lp=0
          for i in range(len(val_data)):
            l=l1*exp(uni_val[i])+l2*exp(bi_val[i])+l3*exp(tri_val[i])+l4*exp(quad_val[i])
            lp+=log(l) if l>0 else 0
          lp/=1000
          if best_log_prob<lp:
            lV1,lV2,lV3,lV4=l1,l2,l3,l4
            best_log_prob=lp
print(lV1,lV2,lV3,lV4)
print(best_log_prob)

0.9 0.0 0.0 0.1
-106.04746956391462
